In [1]:
import torch
from torch.utils.data import DataLoader
from datasets import load_dataset
from transformers import BertTokenizer, BertModel
import torch.nn as nn
import numpy as np
from tqdm import tqdm

# 加载数据集
my_dataset = load_dataset("sst2")

# 加载BERT模型和分词器
model_path = 'model'  # 使用预训练的BERT模型
tokenizer = BertTokenizer.from_pretrained(model_path)
bert_model = BertModel.from_pretrained(model_path)

# 将模型和数据移动到GPU上
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bert_model.to(device)

# 定义情感分类模型
class SentimentClassification(nn.Module):
    def __init__(self, bert_model, hidden_dim, output_dim):
        super().__init__()
        self.bert = bert_model
        self.fc1 = nn.Linear(bert_model.config.hidden_size, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(0.1)

    def forward(self, input_ids, attention_mask):
        # 获取BERT的输出
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        
        # 获取最后一个隐藏层的表示
        pooled_output = outputs.last_hidden_state[:, 0, :]
        
        # 使用全连接层进行分类
        x = self.fc1(pooled_output)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        
        return x

# Tokenize examples
tokenized_dataset = my_dataset.map(lambda example: tokenizer(example["sentence"], padding="max_length", truncation=True), batched=True)

# 切分数据集
train_dataset = tokenized_dataset["train"]
eval_dataset = tokenized_dataset["validation"]
small_train_dataset = tokenized_dataset["train"].shuffle(seed=42).select(range(80))
small_eval_dataset = tokenized_dataset["validation"].shuffle(seed=42).select(range(32))

# 定义数据加载器
batch_size = 2

def collate_fn(batch):
    # 提取输入列表和标签列表
    input_ids_list = [example['input_ids'] for example in batch]
    attention_mask_list = [example['attention_mask'] for example in batch]
    labels_list = [example['label'] for example in batch]
    
    # 转换为张量并删除大小为1的维度
    input_ids = torch.tensor(input_ids_list).squeeze().to(device)
    attention_mask = torch.tensor(attention_mask_list).squeeze().to(device)
    labels = torch.tensor(labels_list).to(device)
    
    return input_ids, attention_mask, labels

# 在数据加载器中使用自定义的collate_fn
train_dataloader = DataLoader(
    small_train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
eval_dataloader = DataLoader(
    small_eval_dataset, batch_size=batch_size, collate_fn=collate_fn)

# 初始化情感分类模型
output_dim = 2  # 二分类任务
hidden_dim = 128  # 隐藏层维度
model = SentimentClassification(bert_model, hidden_dim, output_dim).to(device)

# 定义优化器
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

# 定义损失函数
criterion = nn.CrossEntropyLoss()

# 训练参数设置
num_epochs = 50
eval_steps = len(train_dataloader) // 10

# 模型训练
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)

# 设置训练参数
num_training_steps = len(train_dataloader) * num_epochs
progress_bar = tqdm(range(num_training_steps))
def compute_metrics(model, eval_dataloader):
    model.eval()
    eval_loss = 0
    eval_accuracy = 0
    total_eval_samples = 0
    
    for batch in eval_dataloader:
        with torch.no_grad():
            inputs = {'input_ids': batch[0], 'attention_mask': batch[1]}
            outputs = model(**inputs)
            logits = outputs
            loss = criterion(logits, batch[2])
        
        eval_loss += loss.item()
        logits_np = logits.detach().cpu().numpy()
        predictions = np.argmax(logits_np, axis=-1)
        labels_np = batch[2].cpu().numpy()
        eval_accuracy += np.sum(predictions == labels_np)
        total_eval_samples += len(labels_np)
    
    eval_loss /= len(eval_dataloader)
    eval_accuracy /= total_eval_samples
    return eval_loss, eval_accuracy

# 模型训练
model.train()
for epoch in range(num_epochs):
    print('epoch:', epoch)
    for batch in train_dataloader:
        inputs = {'input_ids': batch[0], 'attention_mask': batch[1]}
        outputs = model(**inputs)
        loss = criterion(outputs, batch[2])
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
    
    # 在评估数据集上计算指标
    eval_loss, eval_accuracy = compute_metrics(model, eval_dataloader)
    print("Eval Loss:", eval_loss)
    print("Eval Accuracy:", eval_accuracy)
    
    # 保存当前模型
    torch.save(model.state_dict(), './checkpoint.pt')

# 加载最佳模型
best_model = SentimentClassification(bert_model, hidden_dim, output_dim).to(device)
best_model.load_state_dict(torch.load('./checkpoint.pt'))


/home/daixin/.local/lib/python3.10/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.25.11) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
/home/daixin/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
  0%|          | 0/2000 [00:00<?, ?it/s]

epoch: 0


  2%|▏         | 40/2000 [00:02<01:48, 18.00it/s]

Eval Loss: 0.6853411234915257
Eval Accuracy: 0.53125


  2%|▏         | 42/2000 [00:04<07:48,  4.17it/s]

...

 98%|█████████▊| 1963/2000 [04:05<00:06,  5.77it/s]

epoch: 49


100%|█████████▉| 1999/2000 [04:08<00:00,  9.24it/s]

Eval Loss: 1.281988362876291
Eval Accuracy: 0.75


<All keys matched successfully>